Get the new articles of 20 minuten
using their rss feed.
Articles are considered as new if published after the last check date


In [84]:
import pandas as pd
import numpy as np
import feedparser
import datetime
import pathlib

In [85]:
# Get feed
url = 'https://partner-feeds.20min.ch/rss/20minuten/zentralschweiz'
feed = feedparser.parse(url)
print(f"Articles in feed : {len(feed.entries)}")

Articles in feed : 31


In [86]:
# Get last check date from global last_check_articles_20min
# liverss_and_coms_path = pathlib.Path.home() / 'Desktop' / 'SocialMediaMonitor' / 'data' / 'raw' / 'liverss_and_coms_20min.csv'
liverss_and_coms_path = '~/Desktop/SocialMediaMonitor/data/raw/liverss_and_coms_20min.csv'
liverss_and_coms_df = pd.read_csv(liverss_and_coms_path)
last_check_articles_20min = pd.to_datetime(liverss_and_coms_df['published'].max())
last_check_articles_20min

Timestamp('2022-05-08 17:31:24+0000', tz='UTC')

In [87]:
# Computing the newly aquired articles -> not used up to now (04.05.2022)
articles_to_keep = []
for entry in feed.entries:
    entry_date = pd.to_datetime(entry.published)
    # print(f"Entry date : {entry_date}")
    if last_check_articles_20min < entry_date:
        articles_to_keep.append(entry)
print(f"New articles in feed since last check : {len(articles_to_keep)}")

New articles in feed since last check : 3


In [88]:
# Compute temporary Pandas dataframe to further process beneath
liverss_tmp_df = pd.DataFrame.from_dict(articles_to_keep, orient='columns')
liverss_tmp_df['published'] = pd.to_datetime(liverss_tmp_df['published'])
liverss_tmp_df.head()

,title,title_detail,links,link,summary,summary_detail,id,guidislink,published,published_parsed
0,Wie sich dieser erfolgreiche Luzerner Radiomac...,"{'type': 'text/plain', 'language': None, 'base...","[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://www.20min.ch/545851604537,Ueli Häfliger hat sich in Luzern einen Namen a...,"{'type': 'text/html', 'language': None, 'base'...",https://www.20min.ch/story/wie-sich-dieser-erf...,False,2022-05-10 02:53:40+00:00,"(2022, 5, 10, 2, 53, 40, 1, 130, 0)"
1,«Polarstar»-Besitzer sucht Nachfolger für Revi...,"{'type': 'text/plain', 'language': None, 'base...","[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://www.20min.ch/730773678727,"Schaumpartys, Fasnachtsbälle, viel Alkohol und...","{'type': 'text/html', 'language': None, 'base'...",https://www.20min.ch/story/polarstar-besitzer-...,False,2022-05-09 17:09:30+00:00,"(2022, 5, 9, 17, 9, 30, 0, 129, 0)"
2,Bei Enkeltrick-Betrügern klickten die Handsche...,"{'type': 'text/plain', 'language': None, 'base...","[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://www.20min.ch/577039134851,Bei einer geplanten Geldübergabe konnten zivil...,"{'type': 'text/html', 'language': None, 'base'...",https://www.20min.ch/story/bei-enkeltrick-betr...,False,2022-05-09 15:44:54+00:00,"(2022, 5, 9, 15, 44, 54, 0, 129, 0)"


In [89]:
# Cleaning created dataframe of 20min liverss_tmp_df
liverss_df = pd.DataFrame()
liverss_df['title'] = liverss_tmp_df['title']
liverss_df['summary'] = liverss_tmp_df['summary']
liverss_df['author'] = ''
liverss_df['is_com_or_sub'] = 'sub'
liverss_df['link'] = liverss_tmp_df['link']
liverss_df['id'] = liverss_tmp_df['id'].str[-12:]
liverss_df['published'] = liverss_tmp_df['published']
liverss_df['published_parsed'] = liverss_tmp_df['published_parsed']
liverss_df.head()


,title,summary,author,is_com_or_sub,link,id,published,published_parsed
0,Wie sich dieser erfolgreiche Luzerner Radiomac...,Ueli Häfliger hat sich in Luzern einen Namen a...,,sub,https://www.20min.ch/545851604537,545851604537,2022-05-10 02:53:40+00:00,"(2022, 5, 10, 2, 53, 40, 1, 130, 0)"
1,«Polarstar»-Besitzer sucht Nachfolger für Revi...,"Schaumpartys, Fasnachtsbälle, viel Alkohol und...",,sub,https://www.20min.ch/730773678727,730773678727,2022-05-09 17:09:30+00:00,"(2022, 5, 9, 17, 9, 30, 0, 129, 0)"
2,Bei Enkeltrick-Betrügern klickten die Handsche...,Bei einer geplanten Geldübergabe konnten zivil...,,sub,https://www.20min.ch/577039134851,577039134851,2022-05-09 15:44:54+00:00,"(2022, 5, 9, 15, 44, 54, 0, 129, 0)"


In [90]:
# Saving generated data to raw data folder
liverss_df.to_csv(r'~/Desktop/SocialMediaMonitor/data/raw/liverss_20Min.csv', index = False)